# Reviews Topic Modeling

In [ ]:
# !pip install -q --disable-pip-version-check -r ../requirements.txt

In [ ]:
# Randomization for seed reproduction
import random

# Matrix operations
import numpy as np

# Dataframes
import pandas as pd

# Regular Expressions
import re

# Import unidecode (to remove accents from words)
from unidecode import unidecode

# Import wordcloud
from wordcloud import WordCloud

# Sklearn Utilities
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation

# Visualize the information contained in a topic model
import pyLDAvis
from pyLDAvis import gensim as gensimvis

import gensim
from gensim import corpora

# Charts
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

# SciKit Learn Utilities
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.manifold import TSNE
# from sklearn.utils import shuffle
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix

# Natural Language Toolkit
import nltk
from nltk.tokenize import word_tokenize

# Language Detection
from langdetect import detect


In [ ]:
# Set a random seed for reproducibility
random_seed = 10
np.random.seed(random_seed)
random.seed(random_seed)

---
## 1. Displaying initial data

In [ ]:
reviews_df = pd.read_csv('../data/Big_AHR.csv', index_col=0)
reviews_df.head()

Obtain an example from the dataset:

In [ ]:
def get_one_sample_with_columns(df, id=0):
    return list(zip(list(df.columns), list(df.iloc[id])))

Display the rating and the sentiment associated to the review:

In [ ]:
def display_ratings(df):
    rating = df['rating'].value_counts()

    fig = px.bar(
        x=rating.index,
        y=rating.values,
        labels={'x': 'Rating', 'y': 'Count'},
        title='Count of Each Unique Rating'
    )

    fig.show()

In [ ]:
def display_review_sentiment(df):
    SENTIMENT_CONVERSION = { 0: "Negative", 1: "Positive", 3: "Neutral" }

    sentiment_column = df['label'].value_counts().sort_index()

    fig = px.bar(
        # Convert numerical x-axis labels to string labels
        x=[SENTIMENT_CONVERSION[label] for label in sentiment_column.index],
        y=sentiment_column.values,
        labels={'x': 'Sentiment', 'y': 'Count'},
        title='Count of Each Unique Sentiment'
    )

    fig.show()

In [ ]:
# display_ratings(reviews_df)

In [ ]:
# display_review_sentiment(reviews_df)

As we can see, the dataset is completely unbalanced. There are almost 10x of 5-star ratings that 2-star ratings.

---
## 2. Text Preprocessing
We will discard everything but the review *review_text* as it is the only useful for the topic modelling process

In [ ]:
# Get every column and drop everything except for review_text
reviews_df = reviews_df.drop(reviews_df.columns.difference(['review_text']), axis=1)

reviews_df.head()

### 2.1. Removing NaN and missing values

In [ ]:
def has_nan_or_missing_values(df):
    # Check for NaN values
    nan_check = df.isna().any()

    # Check for empty values (assuming empty strings)
    empty_check = (df == '').any()

    # Combine the results to check for both NaN and empty values
    has_nan_or_empty = nan_check | empty_check

    print(has_nan_or_empty)
    print(f"Number of samples: {df.shape[0]}")

In [ ]:
has_nan_or_missing_values(reviews_df)

✅ Removing the NaN values resulted on also removing the NaN and missing values from the hotel attribute.

In [ ]:
get_one_sample_with_columns(reviews_df)

Obtain more information about the dataframe:

In [ ]:
reviews_df.info()

### 2.2. Text Preprocessing
We will only keep the Spanish reviews with the langdetect library. Otherwise, the TF_IDF matrix will be too sparse.

#### 2.2.1. Filter Spanish reviews

In [ ]:
# Remove all non-Spanish reviews
# reviews_df = reviews_df[reviews_df['review_text'].apply(lambda x: detect(x) == 'es')]
# reviews_df.info()

In [ ]:
# Reset the indeces
# reviews_df = reviews_df.reset_index(drop=True)

In [ ]:
# Save the dataframe to a CSV file
# reviews_df.to_csv('../data/Topic_Modelling_Big_AHR_Spanish.csv')

In [ ]:
# Import the CSV file
reviews_df = pd.read_csv('../data/Topic_Modelling_Big_AHR_Spanish.csv', index_col=0)

reviews_df.head()

#### 2.2.2. Remove punctuation

In [ ]:
# Remove punctuation using regular expressions
# We just keep the words and spaces
reviews_df['review_text'] = reviews_df['review_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

reviews_df.head()

#### 2.2.3. Transform text to lowercase

In [ ]:
# Transform all words to lowercase (consistency)
reviews_df['review_text'] = reviews_df['review_text'].apply(lambda x: x.lower())

reviews_df.head()

#### 2.2.4. Remove Stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
# Initialize stopwords
stop_words = set(stopwords.words('spanish'))

# Define the text preprocessing function
def remove_stop_words(text):
    # Remove stop words
    words = text.split()  # Split text into words
    words = [word for word in words if word not in stop_words]  # Remove stop words
    return ' '.join(words)

In [ ]:
# preprocess the df
reviews_df['review_text'] = reviews_df['review_text'].apply(remove_stop_words)

reviews_df.head()

#### 2.2.5. Remove non-spanish words
Delete not spanish words or misspelled words that lead to errors and confusion
##### 3 hours to execute, please don't. Instead use the file Reduced_Topic_Modelling_Big_AHR_Spanish.csv

In [ ]:
# from concurrent.futures import ThreadPoolExecutor
# from spellchecker import SpellChecker

# spanish = SpellChecker(language='es')

# def fix_non_words(text: str, column_n) -> str:
#     corrected_words = [spanish.correction(word) if spanish.correction(word) else '' for word in text.split()]
#     # print(f'{text} -> {corrected_words}')
#     return ' '.join(corrected_words)

# # Define the number of threads you want to use
# num_threads = 12

# # Create a ThreadPoolExecutor with the specified number of threads
# with ThreadPoolExecutor(max_workers=num_threads) as executor:
#     # Use the executor to apply the fix_non_words function to each element in the 'review_text' column concurrently
#     reviews_df['review_text'] = list(executor.map(fix_non_words, reviews_df['review_text'], reviews_df.index))


# save the dataframe to a CSV file
# reviews_df.to_csv('../data/Reduced_Topic_Modelling_Big_AHR_Spanish.csv')

In [ ]:
reviews_df = pd.read_csv('../data/Reduced_Topic_Modelling_Big_AHR_Spanish.csv', index_col=0)

reviews_df.head()

#### 2.2.4. Remove accents - deprecated
With the application of the spellchecker, there is no need to remove accents.

As we are processing a latin language, is important to strip accents and special characters in order to generalize better

In [ ]:
# reviews_df['review_text'] = reviews_df['review_text'].apply(unidecode)

# reviews_df.head()

#### 2.2.5. Remove single and pair character words
Problem detected in the exploratory analysis. 

In [ ]:
reviews_df['review_text'] = reviews_df['review_text'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))

---
## 3. Exploratory analysis
Show wordcloud to visualize possible problems in the data

### 3.1. Wordcloud

In [ ]:
def generate_wordcloud(df, column_name, name='wordcloud', random_seed=None):
    # Join all reviews into a single string
    all_reviews = " ".join(df[column_name])

    # Generate the word cloud with the specified random seed
    wordcloud = WordCloud(
        width=1600, height=800, max_font_size=200, background_color="white", colormap="magma",
        random_state=random_seed
    ).generate(all_reviews)

    # Create a Matplotlib figure and axis
    fig, ax = plt.subplots(figsize=(16, 8), facecolor="white")

    # Add the title
    plt.title('Word Cloud for Reviews', fontsize=24, pad=20, color='black', fontweight='medium')

    # Display the word cloud
    ax.imshow(wordcloud, interpolation="bilinear")
    ax.axis("off")

    # Save the generated image
    plt.savefig(f'../images/{name}.png')
    
    # Show the generated image
    plt.show()

In [ ]:
# generate_wordcloud(reviews_df, "review_text", "wordcloud_pre_manual", random_seed)

### 3.2. Manual review

In [ ]:
# Delete words that may be due to errors
wrong_words = ['ma', 'asi', 'una', 'aunque', 'además', 'hace', 'etc', 'q', 'pue', 'si', 'tan', 'sido']
reviews_df['review_text'] = reviews_df['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in wrong_words]))

In [ ]:
generate_wordcloud(reviews_df, "review_text", 'wordcloud_post_manual', random_seed)

---
## 4. Latent Dirichlet Allocation

### 4.1. Tokenize words

In [ ]:
# Download the resources for NLTK spanish tokenizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

In [ ]:
# Tokenize the reviews if they are not already tokenized
tokens_df = reviews_df.copy()

tokens_df['review_text'] = tokens_df['review_text'].apply(lambda x: word_tokenize(x, language='spanish') if isinstance(x, str) else x)

### 4.2. Corpus & Dictionaries

In [ ]:
# Tokenize the reviews if they are not already tokenized
tokens_df["review_text"] = tokens_df["review_text"].apply(
    lambda x: word_tokenize(x, language="spanish") if isinstance(x, str) else x
)

# Create Dictionary
id2word = corpora.Dictionary(tokens_df["review_text"])

# Create Corpus
texts = tokens_df["review_text"]

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

---
## 5. Latent Dirichlet Allocation Training

### 5.1. Output functions

#### 5.1.1. Topic Wordclouds

In [ ]:
def create_topic_wordclouds(lda_model, name, num_topics):
    for topic_num in range(num_topics):
        # Get the words associated with the topic
        topic_words = lda_model.show_topic(topic_num, topn=30)
        # Create a dictionary of topic words and their probabilities
        topic_word_dict = {word: prob for word, prob in topic_words}
        
        # Generate a word cloud for the topic
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(topic_word_dict)
        
        # Display the word cloud for the topic
        plt.figure(figsize=(8, 4))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(f'Topic {topic_num + 1} Word Cloud')
        plt.axis("off")
        plt.savefig(f'../images/topic_{name}_{topic_num + 1}_wordcloud.png')


In [ ]:
import matplotlib.colors as mcolors


def create_multiple_topic_wordclouds(
    lda_model, num_topics, width=2500, height=1800, max_words=20
):
    cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]

    cloud = WordCloud(
        background_color="white",
        width=width,
        height=height,
        max_words=max_words,
        colormap="tab10",
    )

    topics = lda_model.show_topics(formatted=False)

    # Calculate the number of rows for subplots
    num_rows = (num_topics + 1) // 2

    # Create subplots
    fig, axes = plt.subplots(2, 5, figsize=(15, 15), sharex=True, sharey=True)

    for i, ax in enumerate(axes.flatten()):
        if i < num_topics:
            topic_words = dict(topics[i][1])
            cloud.generate_from_frequencies(topic_words, max_font_size=300)
            ax.imshow(cloud)
            ax.set_title("Topic " + str(i), fontdict=dict(size=16))
            ax.axis("off")
        else:
            # Remove extra subplots if there are more than 'num_topics'
            fig.delaxes(ax)

    plt.tight_layout()
    plt.show()

#### 5.1.2. Intertopic Distance Map

In [ ]:
def intertopic_distance(lda_model, name, corpus):
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
  pyLDAvis.save_html(vis, f'../images/{name}.html')
  return vis

### 5.2. LDA Model 1. 10 Topics

Better to use gensim library rather than scykitlearn

In [ ]:
# number of topics
num_topics = 10

# Build LDA model
lda_model1 = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       random_state=100,)
# Print the Keyword in the 10 topics
print(lda_model1.print_topics())
doc_lda = lda_model1[corpus]

#### 5.2.1. Output

In [ ]:
create_topic_wordclouds(lda_model1, 'lda_model1', num_topics)

In [ ]:
# create_multiple_topic_wordclouds(lda_model1, num_topics)

In [ ]:
intertopic_distance(lda_model1, 'lda_model1', corpus)

### 5.3. LDA Model 2. 10 Topics with Parameter Tunning

In [ ]:
# number of topics
num_topics = 10

# Define LDA model parameters
lda_params = {
    'corpus': corpus,
    'id2word': id2word,
    'num_topics': num_topics,
    'random_state': 100,  # Reproducibility seed
    'passes': 30,  # Increase the number of passes for better convergence
    'eta': 'auto',  # Automatically set eta
}

# Build LDA model
lda_model2 = gensim.models.LdaMulticore(**lda_params)

# Print the Keyword in the 10 topics
print(lda_model2.print_topics())
doc_lda = lda_model2[corpus]

#### 5.3.1. Output

In [ ]:
create_topic_wordclouds(lda_model2, 'lda_model2', num_topics)

In [ ]:
# create_multiple_topic_wordclouds(lda_model2, num_topics)

In [ ]:
intertopic_distance(lda_model2, 'lda_model2', corpus)

### 5.4. LDA Model 3. 4 Topics

In [ ]:
# number of topics
num_topics = 4

# Build LDA model
lda_model3 = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       random_state=100,)

# Print the Keyword in the 10 topics
print(lda_model3.print_topics())
doc_lda = lda_model3[corpus]

#### 5.4.1. Output

In [ ]:
create_topic_wordclouds(lda_model3, 'lda_model3', num_topics)

In [ ]:
# create_multiple_topic_wordclouds(lda_model3, num_topics)

In [ ]:
intertopic_distance(lda_model3, 'lda_model3', corpus)

### 5.5. LDA Model 4. 4 Topics with Parameter Tunning

In [ ]:
# number of topics
num_topics = 4

# Define LDA model parameters
lda_params = {
    'corpus': corpus,
    'id2word': id2word,
    'num_topics': num_topics,
    'random_state': 100,  # Reproducibility seed
    'passes': 30,  # Increase the number of passes for better convergence
    'eta': 'auto',  # Automatically set eta
}

# Build LDA model
lda_model4 = gensim.models.LdaMulticore(**lda_params)

# Print the Keyword in the 10 topics
print(lda_model4.print_topics())
doc_lda = lda_model4[corpus]

#### 5.2.1. Output

In [ ]:
create_topic_wordclouds(lda_model4, 'lda_model4', num_topics)

In [ ]:
# create_multiple_topic_wordclouds(lda_model4, num_topics)

In [ ]:
intertopic_distance(lda_model4, 'lda_model4', corpus)

---
## 6. Evaluation

**Coherence Score**: Coherence score quantifies the interpretability and consistency of topics in a topic model; higher coherence scores indicate more meaningful and coherent topics within the model.

**Perplexity**: Perplexity is a measure of how well a probabilistic model, such as a language model or a topic model, predicts a given dataset; lower perplexity values indicate that the model better captures the data's underlying patterns and is more confident in its predictions.

In [ ]:
from gensim.models import CoherenceModel

In [ ]:
# Evaluate Model 1
coherence_model1 = CoherenceModel(model=lda_model1, texts=texts, dictionary=id2word, coherence='c_v')
coherence1 = coherence_model1.get_coherence()
perplexity1 = lda_model1.log_perplexity(corpus)

In [ ]:
# Evaluate Model 2
coherence_model2 = CoherenceModel(model=lda_model2, texts=texts, dictionary=id2word, coherence='c_v')
coherence2 = coherence_model2.get_coherence()
perplexity2 = lda_model2.log_perplexity(corpus)

In [ ]:
# Evaluate Model 3
coherence_model3 = CoherenceModel(model=lda_model3, texts=texts, dictionary=id2word, coherence='c_v')
coherence3 = coherence_model3.get_coherence()
perplexity3 = lda_model3.log_perplexity(corpus)

In [ ]:
# Evaluate Model 4
coherence_model4 = CoherenceModel(model=lda_model4, texts=texts, dictionary=id2word, coherence='c_v')
coherence4 = coherence_model4.get_coherence()
perplexity4 = lda_model4.log_perplexity(corpus)

### 6.1. Coherence scores

In [ ]:
# Coherence scores
print(f"Coherence Score for Model 1: {coherence1}")
print(f"Coherence Score for Model 2: {coherence2}")
print(f"Coherence Score for Model 3: {coherence3}")
print(f"Coherence Score for Model 4: {coherence4}")

### 6.2. Perplexity scores

In [ ]:
# Perplexity scores
print(f"Perplexity for Model 1: {perplexity1}")
print(f"Perplexity for Model 2: {perplexity2}")
print(f"Perplexity for Model 3: {perplexity3}")
print(f"Perplexity for Model 4: {perplexity4}")